# Preprocessing

In [17]:
# Let us start by importing the library from the package directory

import numpy as np 
import pandas as pd 
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
# Read the data
read_data = pd.read_csv("./train.csv")

In [18]:
# Check the first five rows of our dataset
read_data.head()

,id,text,harsh,extremely_harsh,vulgar,threatening,disrespect,targeted_hate
0,a8be7c5d4527adbbf15f,""", 6 December 2007 (UTC)\nI am interested, not...",0,0,0,0,0,0
1,0b7ca73f388222aad64d,I added about three missing parameters to temp...,0,0,0,0,0,0
2,db934381501872ba6f38,SANDBOX?? \n\nI DID YOUR MADRE DID IN THE SANDBOX,1,0,0,0,0,0
3,228015c4a87c4b1f09a7,"why good sir? Why? \n\nYou, sir, obviously do ...",1,0,1,1,1,0
4,b18f26cfa1408b52e949,"""\n\n Source \n\nIncase I forget, or someone e...",0,0,0,0,0,0


### Remove Punctuation

In [19]:
import string

def remove_punctuation(text):
    return "".join([i for i in text if i not in string.punctuation])

read_data['clean_msg']= read_data['text'].apply(lambda x:remove_punctuation(x))

In [20]:
read_data['text'].iloc[0]

'", 6 December 2007 (UTC)\nI am interested, not in arguing, but in the policies which resolve our ongoing content dispute. Also, see Wikipedia: WikiProject United States presidential elections for what I\'ll be working on. Also, the moneybomb closer just self-reverted on two different requests, which echoed what I would have requested.  I will rephrase #3, which I didn\'t see an answer to, building on our agreement that ""moneybomb"" should not be a redlink: Given the deletion reversion, what should be the outline of the article called ""moneybomb"" or should it be submitted for AFD again in due time? (If the latter, see the previous version of #3.) However, this version will require a detailed answer because any ambiguity will only necessitate clarifying questions.   22:32"'

In [21]:
read_data['clean_msg'].iloc[0]

' 6 December 2007 UTC\nI am interested not in arguing but in the policies which resolve our ongoing content dispute Also see Wikipedia WikiProject United States presidential elections for what Ill be working on Also the moneybomb closer just selfreverted on two different requests which echoed what I would have requested  I will rephrase 3 which I didnt see an answer to building on our agreement that moneybomb should not be a redlink Given the deletion reversion what should be the outline of the article called moneybomb or should it be submitted for AFD again in due time If the latter see the previous version of 3 However this version will require a detailed answer because any ambiguity will only necessitate clarifying questions   2232'

In [22]:
read_data['clean_msg']= read_data['clean_msg'].apply(lambda x: x.lower())

In [23]:
read_data['clean_msg'].iloc[0]

' 6 december 2007 utc\ni am interested not in arguing but in the policies which resolve our ongoing content dispute also see wikipedia wikiproject united states presidential elections for what ill be working on also the moneybomb closer just selfreverted on two different requests which echoed what i would have requested  i will rephrase 3 which i didnt see an answer to building on our agreement that moneybomb should not be a redlink given the deletion reversion what should be the outline of the article called moneybomb or should it be submitted for afd again in due time if the latter see the previous version of 3 however this version will require a detailed answer because any ambiguity will only necessitate clarifying questions   2232'

In [27]:
from nltk.tokenize import word_tokenize

read_data['clean_msg'] = read_data.apply(lambda row: word_tokenize(row['clean_msg']), axis=1)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/mayank1609/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************
